# Testing the creation of a 'xr_phenology' function

Random online python phenology functions that might help:

- https://gist.github.com/YanCheng-go/d4e17831f294199443d0f7682558e608

- https://github.com/JavierLopatin/PhenoPY



### Load modules

In [ ]:
import sys
import os
import datacube
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import warnings
import pandas as pd
import deafrica_phenology
sys.path.append('../Scripts')
from deafrica_plotting import display_map, rgb
from deafrica_datahandling import mostcommon_crs, load_ard
from deafrica_bandindices import calculate_indices
from deafrica_phenology import xr_phenology
from deafrica_dask import create_local_dask_cluster

%load_ext autoreload
%autoreload 2

In [ ]:
create_local_dask_cluster(aws_unsigned=False)

### Load some data to work with

In [ ]:
dc = datacube.Datacube(app='phenology stats')

In [ ]:
lat = 22.805
lon = 28.518
# lat = 30.2522
# lon = 30.5516
buffer = 0.0485

x = (lon - buffer, lon + buffer)
y =  (lat + buffer, lat - buffer)

# Create a reusable query
query = {
    'x': x,
    'y': y,
    'time': ('2018-01', '2018-12'),
    'resolution': (-30, 30),
#     'dask_chunks':{'x':500, 'y':500,'time': -1}
}

In [ ]:
# display_map(x=x, y=y)

In [ ]:
# Load available data
ds = load_ard(dc=dc, 
              products=['usgs_ls8c_level2_2'],
              measurements=['red','nir'],
              group_by='solar_day',
              min_gooddata=0.50,
              output_crs='epsg:6933', 
              **query)

# Print output data
print(ds)

### Calculate NDVI and DOY

In [ ]:
# rgb(ds, index=[0,2,4,6,8,10,12,14,16,18,20])

In [ ]:
# First we calculate NDVI on each image in the timeseries
ndvi = calculate_indices(ds, index='NDVI', collection='c2', drop=True)
ndvi = ndvi.NDVI

In [ ]:
ndvi.mean(['x','y']).plot()

### Test each statistic

After confirming each statistic is working, then adding the code to the `deafrica_phenology.py` script.

Statistics to calculate:

    SOS = DOY of start of season
    POS = DOY of peak of season
    EOS = DOY of end of season
    vSOS = Value at start of season
    vPOS = Value at peak of season
    vEOS = Value at end of season
    LOS = Length of season (DOY)
    AOS = Amplitude of season (in value units)
    IOS = Integral of season (SOS-EOS)
    ROG = Rate of greening
    ROS = Rate of senescence
    SW = Skewness of growing season


Statistics that are working:

In [ ]:
%%time
pheno = xr_phenology(ndvi, 
                     interpolate_na=False,
                     interpolate=False,
                     method_sos='median',
                     method_eos='median',
                     interp_method = 'linear',
                     interp_interval = '2W')

print(pheno)

In [ ]:
pheno.Trough.plot(figsize=(10,10))

---

ufunc for applying polynomial fits.  Works okay but is very slow. Better off using the `.resample().interpolate` xarray methods.

In [ ]:
# def poly_fit(time, data, degree):
    
#     pfit = np.polyfit(time, data, degree) 
    
#     return np.transpose(np.polyval(pfit,time))

# def poly_fit_smooth(time, data, degree, n_pts):
#         """
#         """
    
#         time_smooth_inds = np.linspace(0, len(time), n_pts)
#         time_smooth = np.interp(time_smooth_inds, np.arange(len(time)), time)

#         data_smooth = np.array([np.array([coef * (x_val ** current_degree) for
#                                 coef, current_degree in zip(np.polyfit(time, data, degree),
#                                 range(degree, -1, -1))]).sum() for x_val in time_smooth])

#         return data_smooth

# def xr_polyfit(doy,
#                da,
#                degree,
#                interp_multiplier=1):    
    
#     # Fit polynomial curve to observed data points
#     if interp_multiplier==1:
#         print('Fitting polynomial curve to existing observations')
#         pfit = xr.apply_ufunc(
#             poly_fit,
#             doy,
#             da, 
#             kwargs={'degree':degree},
#             input_core_dims=[["time"], ["time"]], 
#             output_core_dims=[['time']],
#             vectorize=True,  
#             dask="parallelized",
#             output_dtypes=[da.dtype],
#         )
    
#     if interp_multiplier > 1:
#         print("Fitting polynomial curve to "+str(len(doy)*interp_multiplier)+
#                                                       " interpolated points")
#         pfit = xr.apply_ufunc(
#             poly_fit_smooth,  # The function
#             doy,# time
#             da,#.chunk({'time': -1}), #the data
#             kwargs={'degree':degree, 'n_pts':len(doy)*interp_multiplier},
#             input_core_dims=[["time"], ["time"]], 
#             output_core_dims=[['new_time']], 
#             output_sizes = ({'new_time':len(doy)*interp_multiplier}),
#             exclude_dims=set(("time",)),
#             vectorize=True, 
#             dask="parallelized",
#             output_dtypes=[da.dtype],
#         ).rename({'new_time':'time'})
    
#         # Map 'dayofyear' onto interpolated time dim
#         time_smooth_inds = np.linspace(0, len(doy), len(doy)*interp_multiplier)
#         new_datetimes = np.interp(time_smooth_inds, np.arange(len(doy)), doy)
#         pfit = pfit.assign_coords({'time':new_datetimes})
    
#     return pfit

# # da=xr_polyfit(dayofyear=dayofyear, 
# #               da=da,
# #               degree=degree,
# #               interp_multiplier=interp_multiplier)
        